In [21]:
import numpy as np
import pandas as pd 
import pytorch_lightning as L
from torchvision import transforms, models,datasets
import cv2
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import Dataset, DataLoader ,random_split,Subset
import matplotlib.pyplot as plt 
import torchvision.models as models
import torch.nn as nn 
import torch.optim as optim 
from torchmetrics import MetricCollection, Accuracy
import torch.nn.functional as F
import torch
import os
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import wandb

In [31]:
# !wandb login 6a66920f640c7001ec17ad4aa7a5da8b378aee61

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [42]:
sweep_config = {
  'name': 'partB',
  'method': 'bayes',
  'metric': {
      'name': 'val_acc',
      'goal': 'maximize'
    },
  'parameters': {
      'epochs': {
            'values': [5]
        },
        'learning_rate': {
            'values': [0.0015,0.0001, 0.01]
        },
        'batch_size': {
            'values': [64]
        },
        'optimizer':{
              'values': ['adam']
        },
        'model':{
            'values':['ResNet50','InceptionV3']
        },
        'unfreeze_layers':{
            'values':[0, 10, 25]
        }
    }
}
sweep_id = wandb.sweep(sweep_config,entity="cs24m019-iitm",project="DL-Assignment-2")

Create sweep with ID: f1b2hsiu
Sweep URL: https://wandb.ai/cs24m019-iitm/DL-Assignment-2/sweeps/f1b2hsiu


In [23]:
class root_dataset(Dataset):
    def __init__(self):
        self.dataset1=datasets.ImageFolder(root='/kaggle/input/inatural-12k/inaturalist_12K/train')
        l1=int(len(self.dataset1)*0.8)
        train_dataset,val_dataset=random_split(self.dataset1, [int(len(self.dataset1)*0.8),len(self.dataset1)-l1])
        #print(len(train_dataset),len(val_dataset))
        self.train_dataset=train_dataset
        self.val_dataset=val_dataset
    def get_train_data(self):
        return self.train_dataset
    def get_val_data(self):
        return self.val_dataset

In [24]:
class inaturalist_train(Dataset):
    def __init__(self,train_data,model_name):
        if model_name=='InceptionV3':
            self.target_size=(3,299,299)
        else:
            self.target_size=(3,224,224)
        #dataset1=datasets.ImageFolder(root='/kaggle/input/neurolist/inaturalist_12K/train')
        self.dataset=train_data    
        self.transform = transforms.Compose([
            transforms.Resize(self.target_size[1:]),
            transforms.ToTensor()
        ])#self.target_size = target_size 
        
    def __getitem__(self,idx):
        image,label=self.dataset[idx]
        image=self.transform(image)
        return image,label
    def __len__(self):
        return len(self.dataset)

In [25]:
class inaturalist_val(Dataset):
    def __init__(self,val_data,model_name):
        if model_name=='InceptionV3':
            self.target_size=(3,299,299)
        else:
            self.target_size=(3,224,224)
        #dataset1=datasets.ImageFolder(root='/kaggle/input/neurolist/inaturalist_12K/train')
        self.dataset=val_data
        self.transform = transforms.Compose([
            transforms.Resize(self.target_size[1:]),
            transforms.ToTensor()
        ])#self.target_size = target_size   
    def __getitem__(self,idx):
        image,label=self.dataset[idx]
        image=self.transform(image)
        return image,label
    def __len__(self):
        return len(self.dataset)

In [26]:
class inaturalist_test(Dataset):
    def __init__(self,model_name):
        if model_name=='InceptionV3':
            self.target_size=(3,299,299)
        else:
            self.target_size=(3,224,224)
        self.dataset=datasets.ImageFolder(root='/kaggle/input/inatural-12k/inaturalist_12K/val')
        self.transform = transforms.Compose([
            transforms.Resize(self.target_size[1:]),  # Resize images to target size
            transforms.ToTensor()
        ])#self.target_size = target_size   
    def __getitem__(self,idx):
        image,label=self.dataset[idx]
        image=self.transform(image)
        return image,label
    def __len__(self):
        return len(self.dataset)

In [27]:
class Activation_Function:
    def activation_Function(self,activation_function):
        if activation_function=='relu':
            return F.relu
        if activation_function=='gelu':
            return F.gelu
        if activation_function=='selu':
            return F.selu
        if activation_function=='elu':
            return F.elu

In [28]:
class lightning_pretrained_CNN(L.LightningModule):
    def __init__(self,model_name,unfreeze_layers,optimizer,learning_rate):
        super().__init__()
        self.learning_rate=learning_rate
        self.optimizer=optimizer
        self.model_name=model_name
        if self.model_name=='ResNet50':
            self.model=models.resnet50(pretrained=True)
            #print('hi')
        if self.model_name=='GoogLeNet':
            self.model=models.googlenet(pretrained=True)
        if self.model_name=='InceptionV3':
            self.model=models.inception_v3(pretrained=True,transform_input=True)
        freeze_index=0
        for param in self.model.parameters():
            if freeze_index< (len(list(self.model.parameters())) - (unfreeze_layers + 2)):
                param.requires_grad = False
            else:
                break
            freeze_index=freeze_index+1
        num_feature=self.model.fc.in_features
        self.model.fc=nn.Linear(num_feature,10)
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        inputs,labels=batch
        output=self.forward(inputs)
        #print(type(output.logits))
        #print(output.shape)
        #print(output.logits)
        if self.model_name=='InceptionV3':
            _,preds = torch.max(output.logits, dim=1)
            loss=F.cross_entropy(output.logits,labels)
        #train_acc = torch.mean(preds == labels)
        #print(pred.shape)
            self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        else:
            _,preds = torch.max(output, dim=1)
            loss=F.cross_entropy(output,labels)
            self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
        return loss
    def configure_optimizers(self):
        if self.optimizer=='adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
            return optimizer
        if self.optimizer=='nadam':
            optimizer = torch.optim.NAdam(self.parameters(), lr=self.learning_rate)
            return optimizer
        if self.optimizer=='sgd':
            optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
            return optimizer
        
    def validation_step(self,batch,batch_idx):
        x, y = batch
        y_pred = self.forward(x)
        val_loss = F.cross_entropy(y_pred, y)
        
        # Compute validation accuracy
        _, predicted = torch.max(y_pred, 1)
        val_acc = torch.sum(predicted == y).item() / y.size(0)
        
        self.log('val_loss', val_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', val_acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
        return val_loss
        
    
    def test_step(self, batch, batch_idx):
        x,y=batch
        pred=self.forward(x)
        loss=F.cross_entropy(pred,y)
        _, predicted = torch.max(pred.data, 1)
        accuracy = torch.sum(predicted == y).item() / y.size(0)
        #print(predicted,accuracy)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_accuracy", accuracy, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"test_loss": loss}
    

In [39]:
def main(config=None):
    with wandb.init(config=config, ):
        config=wandb.config
        wandb.run.name='bs-'+str(config.batch_size)+'-lr-'+ str(config.learning_rate)+'-ep-'+str(config.epochs)+ '-op-'+str(config.optimizer)+'-mn-'+str(config.model)+'-ul-'+str(config.unfreeze_layers)
        model_name=config.model
        root_obj=root_dataset()
        train_data=root_obj.get_train_data()
        val_data=root_obj.get_val_data()
        dataset1=inaturalist_train(train_data,model_name)
        dataset2=inaturalist_val(val_data,model_name)
        dataset3=inaturalist_test(model_name)
        b_size=config.batch_size
        unfreeze_layers=config.unfreeze_layers
        optimizer=config.optimizer
        epoch=config.epochs
        learning_rate=config.learning_rate
        wandb_logger = WandbLogger(project='DL-Assignment-2', entity='cs24m019-iitm')
        dataloader=DataLoader(dataset=dataset1,batch_size=b_size,shuffle=True,num_workers=2)
        val_dataloader=DataLoader(dataset=dataset2,batch_size=b_size,shuffle=False,num_workers=2)
        model=lightning_pretrained_CNN(model_name,unfreeze_layers,'adam',0.0015)
        trainer = L.Trainer(accelerator='auto',devices="auto",max_epochs=epoch,logger=wandb_logger)
        trainer.fit(model,dataloader,val_dataloader)
        test_dataloader=DataLoader(dataset=dataset3,batch_size=8,shuffle=False,num_workers=1)
        trainer.test(dataloaders=test_dataloader)

        

In [43]:
if  __name__ =="__main__":
    wandb.agent(sweep_id, main,count=30)
    #main()

wandb: Agent Starting Run: 95qvmpwp with config:
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0015
wandb: 	model: ResNet50
wandb: 	optimizer: adam
wandb: 	unfreeze_layers: 10


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7680000066757202     │
│         test_loss         │    1.0270122289657593     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▂▂▄▄▅▅▇▇█
test_accuracy,▁
test_loss,▁
train_loss,█▄▂▁▁
trainer/global_step,▁▁▃▃▄▄▆▆███
val_acc,▅██▁▆
val_loss,▁▁▃█▆
epoch,5
test_accuracy,0.768
test_loss,1.02701
train_loss,0.11455
